<a href="https://colab.research.google.com/github/HowardHNguyen/NLP/blob/main/NLP_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [ ]:
corpus = [
          ['money, money, money', 'true'],
          ['free money for free gambling fun', 'true'],
          ['gambling for fun', 'true'],
          ['machine learning for fun, fun, fun', 'false'],
          ['free machine learning', 'false']
]
print(corpus)

[['money, money, money', 'true'], ['free money for free gambling fun', 'true'], ['gambling for fun', 'true'], ['machine learning for fun, fun, fun', 'false'], ['free machine learning', 'false']]


In [ ]:
corpusSpam = [
          ['money, money, money', 'true'],
          ['free money for free gambling fun', 'true'],
          ['gambling for fun', 'true'],
          ['machine learning for fun, fun, fun', 'false'],
          ['free machine learning', 'false']
]
print(corpusSpam)

[['money, money, money', 'true'], ['free money for free gambling fun', 'true'], ['gambling for fun', 'true'], ['machine learning for fun, fun, fun', 'false'], ['free machine learning', 'false']]


In [ ]:
df_corpusSpam = pd.DataFrame(df_corpusSpam, columns=['Sentence', 'Spam'])
print(df_corpusSpam)
totalDocuments = len(df_corpusSpam)
print()
print('Number of Documents = ', totalDocuments)

                             Sentence   Spam
0                 money, money, money   true
1    free money for free gambling fun   true
2                    gambling for fun   true
3  machine learning for fun, fun, fun  false
4               free machine learning  false

Number of Documents =  5


In [ ]:
# Count Vectorizer
X = cv.fit_transform(df_corpusSpam['Sentence'])
print(cv.get_feature_names())
vocabulary = len(cv.get_feature_names())

print('vocabulary size = ', vocabulary)
print(X.shape)
print(X)

['for', 'free', 'fun', 'gambling', 'learning', 'machine', 'money']
vocabulary size =  7
(5, 7)
  (0, 6)	3
  (1, 6)	1
  (1, 1)	2
  (1, 0)	1
  (1, 3)	1
  (1, 2)	1
  (2, 0)	1
  (2, 3)	1
  (2, 2)	1
  (3, 0)	1
  (3, 2)	3
  (3, 5)	1
  (3, 4)	1
  (4, 1)	1
  (4, 5)	1
  (4, 4)	1


In [ ]:
df_all = pd.DataFrame(X.toarray(), columns = cv.get_feature_names())
print(df_all)

   for  free  fun  gambling  learning  machine  money
0    0     0    0         0         0        0      3
1    1     2    1         1         0        0      1
2    1     0    1         1         0        0      0
3    1     0    3         0         1        1      0
4    0     1    0         0         1        1      0


In [ ]:
df_all['Spam'] = df_corpusSpam['Spam']
df_all

,for,free,fun,gambling,learning,machine,money,Spam
0,0,0,0,0,0,0,3,true
1,1,2,1,1,0,0,1,true
2,1,0,1,1,0,0,0,true
3,1,0,3,0,1,1,0,false
4,0,1,0,0,1,1,0,false


In [ ]:
# Compute first level probabilities

p = df_all.where(df_all['Spam'] == 'true').dropna()
# probability of positive documents
p_true = len(p)/totalDocuments
print('First level probability: Positive Documents = ', p_true)

n = df_all.where(df_all['Spam'] == 'false').dropna()
# probability of negative documents
p_false = len(n)/totalDocuments
print('First level probability: Negative Documents = ', p_false)

First level probability: Positive Documents =  0.6
First level probability: Negative Documents =  0.4


In [ ]:
# Positive Count
positive_df = df_all.where(df_all['Spam'] == 'true').dropna()
sum_of_columns = positive_df.sum(axis = 0)

positive_df_sum = positive_df.append(sum_of_columns, ignore_index=True)
positive_df_sum.at[3,'Spam'] = 'SUM true'
print(positive_df_sum)

   for  free  fun  gambling  learning  machine  money      Spam
0  0.0   0.0  0.0       0.0       0.0      0.0    3.0      true
1  1.0   2.0  1.0       1.0       0.0      0.0    1.0      true
2  1.0   0.0  1.0       1.0       0.0      0.0    0.0      true
3  2.0   2.0  2.0       2.0       0.0      0.0    4.0  SUM true


In [ ]:
# Conditional probabilities

totalPositiveWords = 0
for i in range(vocabulary):
  totalPositiveWords = totalPositiveWords + sum_of_columns[i]

conditional_prob = sum_of_columns

for i in range(vocabulary):
  conditional_prob[i] = (conditional_prob[i] + 1) / (vocabulary + totalPositiveWords)

df_pos_sum_prob = positive_df_sum.append(conditional_prob, ignore_index=True)
df_pos_sum_prob.at[4,'Spam'] = 'Probability'
print(df_pos_sum_prob)

        for      free       fun  ...   machine     money         Spam
0  0.000000  0.000000  0.000000  ...  0.000000  3.000000         true
1  1.000000  2.000000  1.000000  ...  0.000000  1.000000         true
2  1.000000  0.000000  1.000000  ...  0.000000  0.000000         true
3  2.000000  2.000000  2.000000  ...  0.000000  4.000000     SUM true
4  0.157895  0.157895  0.157895  ...  0.052632  0.263158  Probability

[5 rows x 8 columns]


In [ ]:
df_pos_sum_prob.T

,0,1,2,3,4
for,0,1,1,2,0.157895
free,0,2,0,2,0.157895
fun,0,1,1,2,0.157895
gambling,0,1,1,2,0.157895
learning,0,0,0,0,0.0526316
machine,0,0,0,0,0.0526316
money,3,1,0,4,0.263158
Spam,true,true,true,SUM true,Probability


In [ ]:
# Conditional Probabilities where Spam is false
negative_df = df_all.where(df_all['Spam'] == 'false').dropna()
sum_of_columns = negative_df.sum(axis = 0)

negative_df_sum = negative_df.append(sum_of_columns, ignore_index=True)
negative_df_sum.at[2,'Spam'] = 'SUM false'
print(negative_df_sum)

   for  free  fun  gambling  learning  machine  money       Spam
0  1.0   0.0  3.0       0.0       1.0      1.0    0.0      false
1  0.0   1.0  0.0       0.0       1.0      1.0    0.0      false
2  1.0   1.0  3.0       0.0       2.0      2.0    0.0  SUM false


In [ ]:
# Conditional probabilities with Spam = false

totalNegativeWords = 0
for i in range(vocabulary):
  totalNegativeWords = totalNegativeWords + sum_of_columns[i]

conditional_prob = sum_of_columns

for i in range(vocabulary):
  conditional_prob[i] = (conditional_prob[i] + 1) / (vocabulary + totalNegativeWords) # +1 is Laplace Smoothing

df_neg_sum_prob = negative_df_sum.append(conditional_prob, ignore_index=True)
df_neg_sum_prob.at[3,'Spam'] = 'Probability'
print(df_neg_sum_prob)

        for      free      fun  ...   machine     money         Spam
0  1.000000  0.000000  3.00000  ...  1.000000  0.000000        false
1  0.000000  1.000000  0.00000  ...  1.000000  0.000000        false
2  1.000000  1.000000  3.00000  ...  2.000000  0.000000    SUM false
3  0.140625  0.140625  0.15625  ...  0.148438  0.132812  Probability

[4 rows x 8 columns]


In [ ]:
df_neg_sum_prob.T

,0,1,2,3
for,1,0,1,0.140625
free,0,1,1,0.140625
fun,3,0,3,0.15625
gambling,0,0,0,0.132812
learning,1,1,2,0.148438
machine,1,1,2,0.148438
money,0,0,0,0.132812
Spam,false,false,SUM false,Probability


In [ ]:
df_pos_sum_prob.T

,0,1,2,3,4
for,0,1,1,2,0.157895
free,0,2,0,2,0.157895
fun,0,1,1,2,0.157895
gambling,0,1,1,2,0.157895
learning,0,0,0,0,0.0526316
machine,0,0,0,0,0.0526316
money,3,1,0,4,0.263158
Spam,true,true,true,SUM true,Probability


In [ ]:
testDoc = ['machine learning for free']
testDoc

['machine learning for free']

In [ ]:
prob_spam_true_test_doc = (0.0526316*0.0526316*0.157895*0.157895)
prob_spam_true_test_doc

6.906052900185266e-05

In [ ]:
prob_spam_false_test_doc = (0.148438*0.148438*0.140625*0.140625)
prob_spam_false_test_doc

0.0004357277898837889